# 2.2. Individual planet distribution with the whole archive

Generate and plot the univariate mass (for Transit case) and bivariate radius-mass (for the RV case) distributions.  
Save results.  

Also do the same with the 8x8 dataset here.

In [1]:
import os
import time
import pickle
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from knnxkde import KNNxKDE
from utils import normalization, renormalization
from utils import convolution_TLG2020_fix_incl

## 0. Prepare matrix

- Load data
- Remove mass if not observed mass
- Select subset of feature for imputation
- Transform log
- Print feature statistics and plot pairplot

In [2]:
original_df = pd.read_csv('data/exoplanets2023.csv')

In [3]:
my_df = original_df.copy()
for n in range(len(my_df)):
    if my_df.loc[n, 'pl_bmassprov'] == 'Msini':
        my_df.loc[n, 'pl_bmassj'] = np.nan

In [4]:
ALL_FEATURES = [
    'pl_radj',
    'pl_bmassj',
    'pl_orbper',
    'pl_orbeccen',
    'pl_orbincl',
    'pl_eqt',
    'st_mass',
    'st_met',
    'st_age',
    'sy_snum',
    'sy_pnum'
]

TLG2020_FEATURES = [
    'pl_radj',
    'pl_bmassj',
    'pl_orbper',
    'pl_eqt',
    'st_mass',
    'sy_pnum',
]

In [5]:
X = np.array(my_df[TLG2020_FEATURES])
X[:, [0, 1, 2, 3, 4]] = np.log(X[:, [0, 1, 2, 3, 4]])

In [6]:
RJ = 11.21  # in Earth radii
MJ = 317.8  # in Earth masses

In [7]:
def generate_planet_bio(n):
    cur_data = my_df.iloc[n]
    pl_rade = cur_data['pl_radj'] * RJ  # radius in Earth radii
    pl_masse = cur_data['pl_bmassj'] * MJ  # mass in Earth masses
    pl_orbper = cur_data['pl_orbper']  # in days
    pl_teq = cur_data['pl_eqt']  # in K
    st_mass = cur_data['st_mass']  # in Solar masses
    pl_pnum = cur_data['sy_pnum']
    my_text = fr'Rad. = {pl_rade:.3f} $r_\oplus$' + '\n'
    my_text += fr'Mass = {pl_masse:.3f} $m_\oplus$' + '\n'
    my_text += fr'OrbPer = {pl_orbper:.2f} days' + '\n'
    my_text += fr'T.Eq. = {pl_teq:.0f} K' + '\n'
    my_text += fr'StMass = {st_mass:.3f} $m_\odot$' + '\n'
    my_text += fr'Nb. Pl. = {pl_pnum}'
    return my_text

## A. Transit Case

In [9]:
DICT_OBSERVED_MASS_NAME_ID = dict()  # store the name and row number of planets with observed mass
for cur_idx in range(my_df.shape[0]):
    if ~np.isnan(my_df['pl_bmassj'].iloc[cur_idx]):
        my_key = my_df['pl_name'].iloc[cur_idx]
        DICT_OBSERVED_MASS_NAME_ID[my_key] = cur_idx
NB_OBSERVED_MASSES = len(DICT_OBSERVED_MASS_NAME_ID)

In [15]:
MY_TAU = 1.0 / 50.0  # for kNNxKDE
MY_NB_NEIGH = 20  # for kNNxKDE
NB_DRAWS = 10000
my_mass_bins = np.geomspace(1e-1, 1e5, num=101)
my_weights = np.ones(NB_DRAWS) / NB_DRAWS
N, D = X.shape

In [17]:
for i, (cur_name, cur_idx)  in enumerate(DICT_OBSERVED_MASS_NAME_ID.items()):
    print(f'{i+1}/{NB_OBSERVED_MASSES}... {cur_name}                      ', end='\r')
    if ((i+1)%100)==0:
        print(f'                                           ', end='\r')
        print(f'{i+1}/{NB_OBSERVED_MASSES} -> {time.strftime("%H:%M:%S", time.localtime())}')
    miss_data = np.copy(X)
    miss_data[cur_idx, 1] = np.nan  # Transit case: hide the mass
    norm_miss_data, norm_params = normalization(miss_data)
    m1 = norm_params['min_val'][1]
    m2 = norm_params['max_val'][1]
    
    knnxkde = KNNxKDE(h=0.05, tau=MY_TAU, nb_neigh=MY_NB_NEIGH, metric='nan_std_eucl')
    knnxkde_samples = knnxkde.impute_samples(norm_miss_data, nb_draws=NB_DRAWS)
    knnxkde_renorm_sample = np.exp(knnxkde_samples[(cur_idx, 1)] * (m2 + 1e-6) + m1) * MJ
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 3))
    true_mass = my_df.iloc[cur_idx]['pl_bmassj'] * MJ
    imputed_mass = np.exp(np.mean(knnxkde_samples[(cur_idx, 1)]) * (m2 + 1e-6) + m1) * MJ
    eps = np.log(true_mass) - np.log(imputed_mass)

    ax.hist(knnxkde_renorm_sample, bins=my_mass_bins, weights=my_weights, color='C3', alpha=0.4)
    ax.axvline(true_mass, ls=(0, (3, 3)), c='black', lw=2)
    ax.axvline(imputed_mass, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${eps:.4f}')
    ax.set_xscale('log')
    ax.set_xlabel('Mass [m$_\oplus$]')
    ax.set_ylabel('Proportion')
    ax.legend()

    props = dict(boxstyle='round', facecolor='white', alpha=0.1)
    ax.text(1.02, 0.98, generate_planet_bio(cur_idx), transform=ax.transAxes, fontsize=10, verticalalignment='top', bbox=props)

    ax.set_title(my_df.iloc[cur_idx]['pl_name'])
    fig.tight_layout()
    plt.savefig(f'results_and_figures/2_comp_whole_archive/distrib_transit/distrib_{str(i).zfill(4)}[{str(cur_idx).zfill(4)}].pdf')
    plt.close()

600/1426 -> 14:20:05                                  
700/1426 -> 14:40:40                           
800/1426 -> 15:01:06                          
900/1426 -> 15:21:37                          
1000/1426 -> 15:42:12                                    
1100/1426 -> 16:02:51                                      
1200/1426 -> 16:23:38                          
1300/1426 -> 16:44:25                                       
1400/1426 -> 17:05:11                                           


## B. RV Case

For the RV case, I already have the distributions.  
I need to perform the convolution, and plot the results.

In [8]:
DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID = dict()  # name and row id for planets with observed mass AND radius
for cur_idx in range(my_df.shape[0]):
    if ~np.isnan(my_df['pl_bmassj'].iloc[cur_idx]) and ~np.isnan(my_df['pl_radj'].iloc[cur_idx]):
        my_key = my_df['pl_name'].iloc[cur_idx]
        DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID[my_key] = cur_idx
NB_OBSERVED_MASS_AND_RADIUS = len(DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID)

In [9]:
MY_TAU = 1.0 / 50.0  # for kNNxKDE
MY_NB_NEIGH = 20  # for kNNxKDE
NB_DRAWS = 10000
my_rad_bins = np.geomspace(1e-1, 1e2, num=101)
my_mass_bins = np.geomspace(1e-1, 1e5, num=101)
my_weights = np.ones(NB_DRAWS) / NB_DRAWS
N, D = X.shape

In [18]:
save_dir = f'results_and_figures/2_comp_whole_archive'  # Reload samples
with open(f'{save_dir}/rv_case_masses_radii.pkl', 'rb') as f:
    imputed_samples = pickle.load(f)  # Note: samples are in log

In [19]:
rad_distrib = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, NB_DRAWS))
mass_distrib = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, NB_DRAWS))
true_radii = np.zeros(NB_OBSERVED_MASS_AND_RADIUS)
true_masses = np.zeros(NB_OBSERVED_MASS_AND_RADIUS)

for i, (cur_name, cur_idx)  in enumerate(DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID.items()):
    rad_distrib[i] = np.exp(imputed_samples[(cur_name, 'radius')])
    mass_distrib[i] = np.exp(imputed_samples[(cur_name, 'mass')])
    true_radii[i] = my_df.iloc[cur_idx]['pl_radj']
    true_masses[i] = my_df.iloc[cur_idx]['pl_bmassj']

In [14]:
nb_repeat = 1000  # Repeat convolution many time
rad_estimates = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, nb_repeat))
mass_estimates = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, nb_repeat))

for n in range(nb_repeat):
    print(f'{n+1}/{nb_repeat}', end='\r', flush=True)
    cur_i = np.arccos(np.random.uniform()) * 90.0 / (np.pi / 2.0)  # random inclination in degrees
    rad_estimates[:, n], mass_estimates[:, n] = convolution_TLG2020_fix_incl(
        rad_distrib=rad_distrib,
        mass_distrib=mass_distrib,
        true_masses=true_masses,
        incl=cur_i,
    )

In [15]:
for i, (cur_name, cur_idx)  in enumerate(DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID.items()):
    print(f'{i+1}/{NB_OBSERVED_MASS_AND_RADIUS}... {cur_name}                      ', end='\r')
    if ((i+1)%100)==0:
        print(f'                                           ', end='\r')
        print(f'{i+1}/{NB_OBSERVED_MASS_AND_RADIUS} -> {time.strftime("%H:%M:%S", time.localtime())}')

    fig, ax = plt.subplots(2, 1, figsize=(8, 5))

    ax[0].hist(rad_distrib[i]*RJ, bins=my_rad_bins, weights=my_weights, color='C3', alpha=0.4)
    rad_estim = np.mean(rad_estimates[i])
    rad_eps = np.log(true_radii[i]) - np.log(rad_estim)
    ax[0].axvline(rad_estim*RJ, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${rad_eps:.4f}')
    ax[0].axvline(true_radii[i]*RJ, ls=(0, (3, 3)), c='black', lw=2)
    ax[0].set_xscale('log')
    ax[0].set_xlabel('Radius [r$_\oplus$]')
    ax[0].set_ylabel('Proportion')
    ax[0].legend()

    ax[1].hist(mass_distrib[i]*MJ, bins=my_mass_bins, weights=my_weights, color='C3', alpha=0.4)
    mass_estim = np.mean(mass_estimates[i])
    mass_eps = np.log(true_masses[i]) - np.log(mass_estim)
    ax[1].axvline(mass_estim*MJ, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${mass_eps:.4f}')
    ax[1].axvline(true_masses[i]*MJ, ls=(0, (3, 3)), c='black', lw=2)
    ax[1].set_xscale('log')
    ax[1].set_xlabel('Mass [m$_\oplus$]')
    ax[1].set_ylabel('Proportion')
    ax[1].legend()

    props = dict(boxstyle='round', facecolor='white', alpha=0.1)
    ax[0].text(1.02, 0.98, generate_planet_bio(cur_idx), transform=ax[0].transAxes, fontsize=10, verticalalignment='top', bbox=props)
    ax[0].set_title(my_df.iloc[cur_idx]['pl_name'])
    fig.tight_layout()
    plt.savefig(f'results_and_figures/2_comp_whole_archive/distrib_rv/distrib_{str(i).zfill(4)}[{str(cur_idx).zfill(4)}].pdf')
    plt.close()

100/1081 -> 09:48:54                                     
200/1081 -> 09:50:35                         
300/1081 -> 09:52:15                           
400/1081 -> 09:53:58                          
500/1081 -> 09:55:37                           
600/1081 -> 09:57:18                          
700/1081 -> 09:59:02                          
800/1081 -> 10:00:47                             
900/1081 -> 10:02:20                              
1000/1081 -> 10:04:07                         


In [10]:
for i, (cur_name, cur_idx)  in enumerate(DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID.items()):
    print(f'{i+1}/{NB_OBSERVED_MASS_AND_RADIUS}... {cur_name}                      ', end='\r')
    if ((i+1)%100)==0:
        print(f'                                           ', end='\r')
        print(f'{i+1}/{NB_OBSERVED_MASS_AND_RADIUS} -> {time.strftime("%H:%M:%S", time.localtime())}')
    miss_data = np.copy(X)
    miss_data[cur_idx, 0] = np.nan  # RV case: hide the radius...
    miss_data[cur_idx, 1] = np.nan  # ... and the mass
    norm_miss_data, norm_params = normalization(miss_data)
    r1 = norm_params['min_val'][0]
    r2 = norm_params['max_val'][0]
    m1 = norm_params['min_val'][1]
    m2 = norm_params['max_val'][1]

    knnxkde = KNNxKDE(h=0.05, tau=MY_TAU, nb_neigh=MY_NB_NEIGH, metric='nan_std_eucl')
    norm_samples = knnxkde.impute_samples(norm_miss_data, nb_draws=NB_DRAWS)
    knnxkde_renorm_rad_sample = np.exp(norm_samples[(cur_idx, 0)] * (r2 + 1e-6) + r1) * RJ
    knnxkde_renorm_mass_sample = np.exp(norm_samples[(cur_idx, 1)] * (m2 + 1e-6) + m1) * MJ

    fig, ax = plt.subplots(2, 1, figsize=(8, 5))
    true_rad = my_df.iloc[cur_idx]['pl_radj'] * RJ
    true_mass = my_df.iloc[cur_idx]['pl_bmassj'] * MJ
    imputed_rad = np.exp(np.mean(knnxkde_samples[(cur_idx, 0)]) * (r2 + 1e-6) + r1) * RJ
    imputed_mass = np.exp(np.mean(knnxkde_samples[(cur_idx, 1)]) * (m2 + 1e-6) + m1) * MJ
    eps_rad = np.log(true_rad) - np.log(imputed_rad)
    eps_mass = np.log(true_mass) - np.log(imputed_mass)

    ax[0].hist(knnxkde_renorm_rad_sample, bins=my_rad_bins, weights=my_weights, color='C3', alpha=0.4)
    ax[0].axvline(true_rad, ls=(0, (3, 3)), c='black', lw=2)
    ax[0].axvline(imputed_rad, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${eps_rad:.4f}')
    ax[0].set_xscale('log')
    ax[0].set_xlabel('Radius [r$_\oplus$]')
    ax[0].set_ylabel('Proportion')
    ax[0].legend()
    
    ax[1].hist(knnxkde_renorm_mass_sample, bins=my_mass_bins, weights=my_weights, color='C3', alpha=0.4)
    ax[1].axvline(true_mass, ls=(0, (3, 3)), c='black', lw=2)
    ax[1].axvline(imputed_mass, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${eps_mass:.4f}')
    ax[1].set_xscale('log')
    ax[1].set_xlabel('Mass [m$_\oplus$]')
    ax[1].set_ylabel('Proportion')
    ax[1].legend()
    
    props = dict(boxstyle='round', facecolor='white', alpha=0.1)
    ax.text(1.02, 0.98, generate_planet_bio(cur_idx), transform=ax.transAxes, fontsize=10, verticalalignment='top', bbox=props)

    ax[0].set_title(my_df.iloc[cur_idx]['pl_name'])
    fig.tight_layout()
    plt.savefig(f'results_and_figures/2_comp_whole_archive/distrib_rv/distrib_{str(i).zfill(4)}[{str(cur_idx).zfill(4)}].pdf')
    plt.close()

100/1081 -> 16:45:38                                     
200/1081 -> 17:03:39                         
300/1081 -> 17:21:39                           
400/1081 -> 17:39:41                          
500/1081 -> 17:57:44                           
600/1081 -> 18:15:45                          
700/1081 -> 18:33:46                          
800/1081 -> 18:51:47                             
900/1081 -> 19:10:01                              
1000/1081 -> 19:28:03                         


# NEW -- When 8 parameters are used

## A. Transit

In [19]:
def generate_planet_bio(n):
    cur_data = my_df.iloc[n]
    pl_rade = cur_data['pl_radj'] * RJ  # radius in Earth radii
    pl_masse = cur_data['pl_bmassj'] * MJ  # mass in Earth masses
    pl_orbper = cur_data['pl_orbper']  # in days
    pl_orbeccen = cur_data['pl_orbeccen']  # e \in [0, 1]
    pl_teq = cur_data['pl_eqt']  # in K
    st_mass = cur_data['st_mass']  # in Solar masses
    st_met = cur_data['st_met']  # in dex
    pl_pnum = cur_data['sy_pnum']
    my_text = fr'Rad. = {pl_rade:.3f} $r_\oplus$' + '\n'
    my_text += fr'Mass = {pl_masse:.3f} $m_\oplus$' + '\n'
    my_text += fr'OrbPer = {pl_orbper:.2f} days' + '\n'
    my_text += fr'OrbEcc. = {pl_orbeccen:.3f}' + '\n'
    my_text += fr'T.Eq. = {pl_teq:.0f} K' + '\n'
    my_text += fr'StMass = {st_mass:.3f} $m_\odot$' + '\n'
    my_text += fr'StMet. = {st_met:.3f} dex' + '\n'
    my_text += fr'Nb. Pl. = {pl_pnum}'
    return my_text

In [20]:
original_df = pd.read_csv('data/exoplanets2023.csv')

my_df = original_df.copy()
for n in range(len(my_df)):
    if my_df.loc[n, 'pl_bmassprov'] == 'Msini':
        my_df.loc[n, 'pl_bmassj'] = np.nan
    if my_df.loc[n, 'pl_orbeccen']<0.0:
        my_df.loc[n, 'pl_orbeccen'] = np.nan  # Remove the 3 suspicious values 

In [21]:
ALL_FEATURES = [
    'pl_radj',
    'pl_bmassj',
    'pl_orbper',
    'pl_orbeccen',
    'pl_orbincl',
    'pl_eqt',
    'st_mass',
    'st_met',
    'st_age',
    'sy_snum',
    'sy_pnum'
]

NEW_FEATURES = [
    'pl_radj',
    'pl_bmassj',
    'pl_orbper',
    'pl_orbeccen',
    'pl_eqt',
    'st_mass',
    'st_met',
    'sy_pnum',
]

In [22]:
X = np.array(my_df[NEW_FEATURES])
X[:, [0, 1, 2, 4, 5]] = np.log(X[:, [0, 1, 2, 4, 5]])

In [25]:
MY_TAU = 1.0 / 50.0  # for kNNxKDE
MY_NB_NEIGH = 20  # for kNNxKDE
NB_DRAWS = 10000
my_mass_bins = np.geomspace(1e-1, 1e5, num=101)
my_weights = np.ones(NB_DRAWS) / NB_DRAWS
N, D = X.shape

In [28]:
for i, (cur_name, cur_idx)  in enumerate(DICT_OBSERVED_MASS_NAME_ID.items()):
    if i<1300:
        continue
    print(f'{i+1}/{NB_OBSERVED_MASSES}... {cur_name}                      ', end='\r')
    if ((i+1)%10)==0:
        print(f'                                           ', end='\r')
        print(f'{i+1}/{NB_OBSERVED_MASSES} -> {time.strftime("%H:%M:%S", time.localtime())}')
    miss_data = np.copy(X)
    miss_data[cur_idx, 1] = np.nan  # Transit case: hide the mass
    norm_miss_data, norm_params = normalization(miss_data)
    m1 = norm_params['min_val'][1]
    m2 = norm_params['max_val'][1]
    
    knnxkde = KNNxKDE(h=0.05, tau=MY_TAU, nb_neigh=MY_NB_NEIGH, metric='nan_std_eucl')
    knnxkde_samples = knnxkde.impute_samples(norm_miss_data, nb_draws=NB_DRAWS)
    knnxkde_renorm_sample = np.exp(knnxkde_samples[(cur_idx, 1)] * (m2 + 1e-6) + m1) * MJ
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 3))
    true_mass = my_df.iloc[cur_idx]['pl_bmassj'] * MJ
    imputed_mass = np.exp(np.mean(knnxkde_samples[(cur_idx, 1)]) * (m2 + 1e-6) + m1) * MJ
    eps = np.log(true_mass) - np.log(imputed_mass)

    ax.hist(knnxkde_renorm_sample, bins=my_mass_bins, weights=my_weights, color='C3', alpha=0.4)
    ax.axvline(true_mass, ls=(0, (3, 3)), c='black', lw=2)
    ax.axvline(imputed_mass, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${eps:.4f}')
    ax.set_xscale('log')
    ax.set_xlabel('Mass [m$_\oplus$]')
    ax.set_ylabel('Proportion')
    ax.legend()

    props = dict(boxstyle='round', facecolor='white', alpha=0.1)
    ax.text(1.02, 0.98, generate_planet_bio(cur_idx), transform=ax.transAxes, fontsize=10, verticalalignment='top', bbox=props)

    ax.set_title(my_df.iloc[cur_idx]['pl_name'])
    fig.tight_layout()
    plt.savefig(f'results_and_figures/2_comp_whole_archive/distrib_transit_all8params/distrib_{str(i).zfill(4)}[{str(cur_idx).zfill(4)}].pdf')
    plt.close()

1310/1426 -> 10:51:49                          
1320/1426 -> 10:55:09                        
1330/1426 -> 10:58:18                       
1340/1426 -> 11:01:22                       
1350/1426 -> 11:04:16                       
1360/1426 -> 11:07:07                       
1370/1426 -> 11:10:09                         
1380/1426 -> 11:13:12                         
1390/1426 -> 11:16:24                         
1400/1426 -> 11:19:22                                           
1410/1426 -> 11:22:11                             
1420/1426 -> 11:25:03                         


## B. RV case

In [8]:
DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID = dict()  # name and row id for planets with observed mass AND radius
for cur_idx in range(my_df.shape[0]):
    if ~np.isnan(my_df['pl_bmassj'].iloc[cur_idx]) and ~np.isnan(my_df['pl_radj'].iloc[cur_idx]):
        my_key = my_df['pl_name'].iloc[cur_idx]
        DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID[my_key] = cur_idx
NB_OBSERVED_MASS_AND_RADIUS = len(DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID)

In [9]:
MY_TAU = 1.0 / 50.0  # for kNNxKDE
MY_NB_NEIGH = 20  # for kNNxKDE
NB_DRAWS = 10000
my_rad_bins = np.geomspace(1e-1, 1e2, num=101)
my_mass_bins = np.geomspace(1e-1, 1e5, num=101)
my_weights = np.ones(NB_DRAWS) / NB_DRAWS
N, D = X.shape

In [12]:
save_dir = f'results_and_figures/2_comp_whole_archive'  # Reload samples
with open(f'{save_dir}/rv_case_masses_radii_8params_nbn20_subset6.pkl', 'rb') as f:
    imputed_samples = pickle.load(f)  # Note: samples are in log

In [13]:
rad_distrib = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, NB_DRAWS))
mass_distrib = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, NB_DRAWS))
true_radii = np.zeros(NB_OBSERVED_MASS_AND_RADIUS)
true_masses = np.zeros(NB_OBSERVED_MASS_AND_RADIUS)

for i, (cur_name, cur_idx)  in enumerate(DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID.items()):
    rad_distrib[i] = np.exp(imputed_samples[(cur_name, 'radius')])
    mass_distrib[i] = np.exp(imputed_samples[(cur_name, 'mass')])
    true_radii[i] = my_df.iloc[cur_idx]['pl_radj']
    true_masses[i] = my_df.iloc[cur_idx]['pl_bmassj']

In [14]:
nb_repeat = 1000  # Repeat convolution many time
rad_estimates = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, nb_repeat))
mass_estimates = np.zeros((NB_OBSERVED_MASS_AND_RADIUS, nb_repeat))

for n in range(nb_repeat):
    print(f'{n+1}/{nb_repeat}', end='\r', flush=True)
    cur_i = np.arccos(np.random.uniform()) * 90.0 / (np.pi / 2.0)  # random inclination in degrees
    rad_estimates[:, n], mass_estimates[:, n] = convolution_TLG2020_fix_incl(
        rad_distrib=rad_distrib,
        mass_distrib=mass_distrib,
        true_masses=true_masses,
        incl=cur_i,
    )

In [15]:
for i, (cur_name, cur_idx)  in enumerate(DICT_OBSERVED_MASS_AND_RADIUS_NAME_ID.items()):
    print(f'{i+1}/{NB_OBSERVED_MASS_AND_RADIUS}... {cur_name}                      ', end='\r')
    if ((i+1)%100)==0:
        print(f'                                           ', end='\r')
        print(f'{i+1}/{NB_OBSERVED_MASS_AND_RADIUS} -> {time.strftime("%H:%M:%S", time.localtime())}')

    fig, ax = plt.subplots(2, 1, figsize=(8, 5))

    ax[0].hist(rad_distrib[i]*RJ, bins=my_rad_bins, weights=my_weights, color='C3', alpha=0.4)
    rad_estim = np.mean(rad_estimates[i])
    rad_eps = np.log(true_radii[i]) - np.log(rad_estim)
    ax[0].axvline(rad_estim*RJ, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${rad_eps:.4f}')
    ax[0].axvline(true_radii[i]*RJ, ls=(0, (3, 3)), c='black', lw=2)
    ax[0].set_xscale('log')
    ax[0].set_xlabel('Radius [r$_\oplus$]')
    ax[0].set_ylabel('Proportion')
    ax[0].legend()

    ax[1].hist(mass_distrib[i]*MJ, bins=my_mass_bins, weights=my_weights, color='C3', alpha=0.4)
    mass_estim = np.mean(mass_estimates[i])
    mass_eps = np.log(true_masses[i]) - np.log(mass_estim)
    ax[1].axvline(mass_estim*MJ, ls=(0, (3, 3)), c='C3', lw=2, label=f'$\\varepsilon=${mass_eps:.4f}')
    ax[1].axvline(true_masses[i]*MJ, ls=(0, (3, 3)), c='black', lw=2)
    ax[1].set_xscale('log')
    ax[1].set_xlabel('Mass [m$_\oplus$]')
    ax[1].set_ylabel('Proportion')
    ax[1].legend()

    props = dict(boxstyle='round', facecolor='white', alpha=0.1)
    ax[0].text(1.02, 0.98, generate_planet_bio(cur_idx), transform=ax[0].transAxes, fontsize=10, verticalalignment='top', bbox=props)
    ax[0].set_title(my_df.iloc[cur_idx]['pl_name'])
    fig.tight_layout()
    plt.savefig(f'results_and_figures/2_comp_whole_archive/distrib_rv_all8params/distrib_{str(i).zfill(4)}[{str(cur_idx).zfill(4)}].pdf')
    plt.close()

100/1081 -> 16:47:45                                     
200/1081 -> 16:49:36                         
300/1081 -> 16:51:28                           
400/1081 -> 16:53:18                          
500/1081 -> 16:55:08                           
600/1081 -> 16:57:00                          
700/1081 -> 16:58:53                          
800/1081 -> 17:00:47                             
900/1081 -> 17:02:33                              
1000/1081 -> 17:04:29                         
